In [1]:
!git clone https://github.com/ayamohamdd/IP-25.git


Cloning into 'IP-25'...
remote: Enumerating objects: 709, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 709 (delta 0), reused 3 (delta 0), pack-reused 700 (from 1)
Receiving objects: 100% (709/709), 403.03 MiB | 49.45 MiB/s, done.
Updating files: 100% (691/691), done.


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, BatchNormalization, Activation, UpSampling2D, Softmax,Dropout
from tensorflow.keras.optimizers import Adam , SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img
import numpy as np
from PIL import Image, ImageEnhance
import matplotlib.pyplot as plt
import os
import cv2

In [3]:
# Paths to the directories
train_image_paths = r"/kaggle/working/IP-25/train_data/Images"
train_mask_paths = r"/kaggle/working/IP-25/train_data/Labels"

# Output paths
output_image_path = train_image_paths
output_mask_path = train_mask_paths

# Image augmentation setup
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Get lists of image and mask files
image_files = sorted([file for file in os.listdir(train_image_paths) if file.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'gif', 'tiff'))])
mask_files = sorted([file for file in os.listdir(train_mask_paths) if file.lower().endswith(('jpg', 'jpeg', 'png', 'bmp', 'gif', 'tiff'))])

# Ensure that image and mask filenames correspond (e.g., "image1.jpg" matches "mask1.jpg")
assert len(image_files) == len(mask_files), "Mismatch in the number of images and masks!"
for img_file, mask_file in zip(image_files, mask_files):
    assert os.path.splitext(img_file)[0] == os.path.splitext(mask_file)[0], "Image and mask filenames do not match!"

# Augment each image-mask pair
counter = 0
for img_file, mask_file in zip(image_files, mask_files):
    # Load the image and mask
    img_path = os.path.join(train_image_paths, img_file)
    mask_path = os.path.join(train_mask_paths, mask_file)
    img = img_to_array(load_img(img_path))
    mask = img_to_array(load_img(mask_path, color_mode="rgb"))  # Ensure mask is grayscale

    # Expand dimensions to match the generator input format
    img = np.expand_dims(img, axis=0)
    mask = np.expand_dims(mask, axis=0)

    # Create a combined generator for both image and mask
    seed = np.random.randint(0, 10000)  # Use the same seed for both
    aug_img_iter = datagen.flow(img, batch_size=1, seed=seed)
    aug_mask_iter = datagen.flow(mask, batch_size=1, seed=seed)

    # Generate augmented images and masks
    for i in range(3):  # Create 3 augmentations per image-mask pair
        aug_img = next(aug_img_iter)[0].astype('uint8')  # Augmented image
        aug_mask = next(aug_mask_iter)[0].astype('uint8')  # Augmented mask

        # Save augmented image and mask with unique names
        img_name = f"aug_{counter}_{img_file}"
        mask_name = f"aug_{counter}_{mask_file}"
        save_img(os.path.join(output_image_path, img_name), aug_img)
        save_img(os.path.join(output_mask_path, mask_name), aug_mask)
        counter += 1

print(f"Data augmentation complete! Total augmented pairs: {counter}")

Data augmentation complete! Total augmented pairs: 600


In [4]:
# import os
# import matplotlib.pyplot as plt
# from tensorflow.keras.preprocessing.image import load_img, img_to_array

# # Paths to the directories with augmented images and masks
# augmented_image_path = r"/kaggle/working/IP-25/train_data/Images"
# augmented_mask_path = r"/kaggle/working/IP-25/train_data/Labels"

# # Get lists of augmented image and mask files
# augmented_image_files = sorted([file for file in os.listdir(augmented_image_path) if file.startswith("aug_")])
# augmented_mask_files = sorted([file for file in os.listdir(augmented_mask_path) if file.startswith("aug_")])

# # Ensure that image and mask filenames correspond
# assert len(augmented_image_files) == len(augmented_mask_files), "Mismatch in the number of augmented images and masks!"
# for img_file, mask_file in zip(augmented_image_files, augmented_mask_files):
#     assert os.path.splitext(img_file)[0] == os.path.splitext(mask_file)[0], "Augmented image and mask filenames do not match!"

# # Display the augmented images and their corresponding masks
# for img_file, mask_file in zip(augmented_image_files, augmented_mask_files):
#     # Load the augmented image and mask
#     img_path = os.path.join(augmented_image_path, img_file)
#     mask_path = os.path.join(augmented_mask_path, mask_file)
#     img = img_to_array(load_img(img_path))  # Load augmented image
#     mask = img_to_array(load_img(mask_path, color_mode="rgb")).squeeze()  # Load augmented mask

#     # Plot the image and corresponding mask
#     plt.figure(figsize=(8, 4))
#     plt.subplot(1, 2, 1)
#     plt.imshow(img.astype('uint8'))
#     plt.title(f"Augmented Image: {img_file}")
#     plt.axis("off")
    
#     plt.subplot(1, 2, 2)
#     plt.imshow(mask)
#     plt.title(f"Augmented Mask: {mask_file}")
#     plt.axis("off")
    
#     plt.show()  # Show the plots

In [4]:
def load_images(image_paths, target_size=(256, 256)):
    images = []
    for img_path in image_paths:
        # Load image
        img = load_img(img_path, target_size=target_size)
        img = img_to_array(img) / 255.0  # Normalize images to [0, 1]

        # Convert to uint8 for OpenCV processing
        img = (img * 255).astype(np.uint8)

        # Decrease brightness
        img = cv2.convertScaleAbs(img, alpha=0.8, beta=0)  # Reduce brightness by 20%

        # Apply Gaussian blur
        img = cv2.GaussianBlur(img, (5, 5), 0)

        # Normalize back to [0, 1] after processing
        img = img.astype(np.float32) / 255.0

        images.append(img)
    return np.array(images)

def rgb_to_class_index(mask_rgb):
    color_map = {
        (0, 0, 0): 0,         # Background clutter
        (128, 0, 0): 1,       # Building
        (128, 64, 128): 2,    # Road
        (0, 128, 0): 3,       # Tree
        (128, 128, 0): 4,     # Low vegetation
        (64, 0, 128): 5,      # Moving car
        (192, 0, 192): 6,     # Static car
        (64, 64, 0): 7        # Human
    }

    min_distance = float('inf')
    closest_color = None
    for color, index in color_map.items():
        distance = np.linalg.norm(np.array(mask_rgb) - np.array(color))  # Euclidean distance
        if distance < min_distance:
            min_distance = distance
            closest_color = index

    return closest_color

In [6]:
def load_masks(mask_paths, target_size=(256, 256)):
    masks = []
    for mask_path in mask_paths:
        mask = load_img(mask_path, target_size=target_size)
        mask = img_to_array(mask)
        mask_class_indices = np.apply_along_axis(rgb_to_class_index, 2, mask.astype(int))
        masks.append(mask_class_indices)
    return np.array(masks)

In [11]:
def segnet_model(input_size=(256, 256, 3), num_classes=8, dropout_rate=0.5):
    inputs = Input(input_size)

    # Encoder
    conv1 = Conv2D(64, (3, 3), padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(64, (3, 3), padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, (3, 3), padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(128, (3, 3), padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, (3, 3), padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(256, (3, 3), padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(256, (3, 3), padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Decoder
    up3 = UpSampling2D(size=(2, 2))(pool3)
    conv4 = Conv2D(256, (3, 3), padding='same')(up3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(256, (3, 3), padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(128, (3, 3), padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)

    up2 = UpSampling2D(size=(2, 2))(conv4)
    conv5 = Conv2D(128, (3, 3), padding='same')(up2)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(64, (3, 3), padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)

    up1 = UpSampling2D(size=(2, 2))(conv5)
    conv6 = Conv2D(64, (3, 3), padding='same')(up1)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(64, (3, 3), padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)

    # Dropout layer
    conv6 = Dropout(dropout_rate)(conv6)

    # Output layer
    output = Conv2D(num_classes, (1, 1), activation='softmax')(conv6)

    model = Model(inputs, output)
    return model

In [12]:
def rand_index(y_true, y_pred):
    y_pred_classes = tf.argmax(y_pred, axis=-1)  # Convert probabilities to class indices
    y_true_flat = tf.keras.backend.flatten(y_true)  # Flatten ground truth
    y_pred_flat = tf.keras.backend.flatten(y_pred_classes)  # Flatten predictions
    equal = tf.reduce_sum(tf.cast(tf.equal(y_true_flat, y_pred_flat), tf.float32))  # Count equal pairs
    total_pairs = tf.cast(tf.size(y_true_flat, out_type=tf.int32), tf.float32)  # Get total number of elements
    return equal / (total_pairs + tf.keras.backend.epsilon())  # Avoid divide by zero


# Jaccard Index (IoU) - Adjusted for probabilities
def jaccard_index(y_true, y_pred):
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])
    y_pred_one_hot = tf.one_hot(y_pred_classes, depth=tf.shape(y_pred)[-1])
    intersection = tf.reduce_sum(y_true_one_hot * y_pred_one_hot, axis=[1, 2])
    union = tf.reduce_sum(y_true_one_hot + y_pred_one_hot, axis=[1, 2]) - intersection
    return tf.reduce_mean(intersection / (union + tf.keras.backend.epsilon()))

# Precision
def precision(y_true, y_pred):
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])
    y_pred_one_hot = tf.one_hot(y_pred_classes, depth=tf.shape(y_pred)[-1])
    true_positives = tf.reduce_sum(y_true_one_hot * y_pred_one_hot, axis=[1, 2])
    predicted_positives = tf.reduce_sum(y_pred_one_hot, axis=[1, 2])
    return tf.reduce_mean(true_positives / (predicted_positives + tf.keras.backend.epsilon()))

# Recall
def recall(y_true, y_pred):
    y_pred_classes = tf.argmax(y_pred, axis=-1)
    y_true_one_hot = tf.one_hot(tf.cast(y_true, tf.int32), depth=tf.shape(y_pred)[-1])
    y_pred_one_hot = tf.one_hot(y_pred_classes, depth=tf.shape(y_pred)[-1])
    true_positives = tf.reduce_sum(y_true_one_hot * y_pred_one_hot, axis=[1, 2])
    actual_positives = tf.reduce_sum(y_true_one_hot, axis=[1, 2])
    return tf.reduce_mean(true_positives / (actual_positives + tf.keras.backend.epsilon()))

In [13]:
model = segnet_model(input_size=(256, 256, 3), num_classes=8)
model.compile(
    optimizer=Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', rand_index, jaccard_index, precision, recall]
)
model2 = segnet_model(input_size=(256, 256, 3), num_classes=8)
model2.compile(
    optimizer=SGD(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy', rand_index, jaccard_index, precision, recall]
)

In [ ]:
def get_image_paths(directory, extensions=[".jpg", ".png"]):
    return [os.path.join(directory, filename) for filename in os.listdir(directory) if any(filename.endswith(ext) for ext in extensions)]

# Paths for training and validation data
train_image_paths = get_image_paths(r"/kaggle/working/IP-25/train_data/Images")
train_mask_paths = get_image_paths(r"/kaggle/working/IP-25/train_data/Labels")
val_image_paths = get_image_paths(r"/kaggle/working/IP-25/val_data/Images")
val_mask_paths = get_image_paths(r"/kaggle/working/IP-25/val_data/Labels")

# Load data
train_images = load_images(train_image_paths)
train_masks = load_masks(train_mask_paths)
val_images = load_images(val_image_paths)
val_masks = load_masks(val_mask_paths)

# Train the model
history = model.fit(
    train_images,
    train_masks,
    validation_data=(val_images, val_masks),
    epochs=80,
    batch_size=16
)
history2 = model2.fit(
    train_images,
    train_masks,
    validation_data=(val_images, val_masks),
    epochs=80,
    batch_size=16
)